# 📊 Exploratory Data Analysis - Basic

Questo notebook esegue un'analisi esplorativa **di base** del dataset per il progetto StiMatrix.

**Nota**: Questo notebook utilizza il modulo `eda_utils.py` per funzioni comuni e best practices.

## 🎯 Obiettivi dell'analisi:
1. Caricamento configurazione e dati
2. Overview del dataset (shape, dtypes, memoria)
3. Analisi missingness
4. Distribuzione del target `AI_Prezzo_Ridistribuito`
5. Correlazioni con il target (top features)
6. Visualizzazioni base
7. Summary per gruppi chiave

## 💡 Per analisi avanzate
Vedi il notebook `eda_advanced.ipynb` per:
- Analisi multi-target
- Correlazioni avanzate (Spearman, Kendall, Cramér's V)
- Feature importance comparativa
- Matrici di correlazione complete

## 1️⃣ Setup e Import

In [ ]:
# Import delle librerie necessarie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Import del modulo utilities
from eda_utils import (
    setup_plotting_style,
    setup_output_dir,
    load_config_and_data,
    get_target_column,
    print_dataset_summary,
    analyze_missingness,
    analyze_target_distribution,
    analyze_correlations,
    plot_target_distribution,
    create_correlation_heatmap,
    save_plot
)

warnings.filterwarnings('ignore')

# Setup plotting style
setup_plotting_style()

print("✅ Setup completato! Modulo eda_utils caricato.")

## 2️⃣ Caricamento Dati

In [ ]:
# Setup output directory
output_dir = setup_output_dir('eda_outputs')

# Carica configurazione e dati
config, df = load_config_and_data(
    config_path='../config/config.yaml',
    data_path='../data/raw/raw.parquet'
)

# Identifica target
target_col = get_target_column(config)

## 3️⃣ Overview Dataset

In [ ]:
# Summary completo del dataset
print_dataset_summary(df)

# Prime righe
print("\n🔍 Prime 5 righe del dataset:")
df.head()

In [ ]:
# Tipi di dati e info dettagliate
print("\n📋 Distribuzione tipi di dati:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  {dtype}: {count} colonne")

# Info su colonne con valori mancanti
print("\n❓ Top 10 colonne per valori mancanti:")
missing_summary = pd.DataFrame({
    'Missing': df.isnull().sum(),
    'Missing %': (df.isnull().mean() * 100).round(2)
})
missing_summary[missing_summary['Missing'] > 0].sort_values('Missing', ascending=False).head(10)

## 4️⃣ Analisi Missingness

In [ ]:
# Analisi completa della missingness
missing_stats = analyze_missingness(df, output_dir)

# Visualizza colonne con alta missingness
high_missing = missing_stats[
    missing_stats['Percentuale_Null'] > 50
].sort_values('Percentuale_Null', ascending=False)

if len(high_missing) > 0:
    print(f"\n⚠️  {len(high_missing)} colonne con >50% valori mancanti")
    high_missing.head(10)
else:
    print("\n✅ Nessuna colonna con >50% valori mancanti")

## 5️⃣ Analisi Target

In [ ]:
# Analisi distribuzione target
stats, price_bands = analyze_target_distribution(df, target_col, output_dir)

# Visualizza statistiche
if stats is not None:
    print("\n📊 Statistiche Target:")
    print(f"  Conteggio: {stats['count']:,.0f}")
    print(f"  Media: €{stats['mean']:,.2f}")
    print(f"  Mediana: €{stats['50%']:,.2f}")
    print(f"  Std Dev: €{stats['std']:,.2f}")
    print(f"  Range: €{stats['min']:,.2f} - €{stats['max']:,.2f}")

In [ ]:
# Visualizzazione distribuzione target
plot_target_distribution(
    df, 
    target_col, 
    output_dir, 
    save_filename='target_distribution'
)

print("✅ Grafici distribuzione salvati in eda_outputs/")

### 5.3 Trasformazioni e Test di Normalità

Analizziamo diverse trasformazioni del target per valutare quale migliora la normalità della distribuzione.

In [ ]:
# Analisi approfondita con trasformazioni
from eda_utils import analyze_target_with_transformations

transform_results = analyze_target_with_transformations(
    df,
    target_col,
    output_dir
)

# Visualizza le trasformazioni
from eda_utils import plot_target_distribution

plot_target_distribution(
    df,
    target_col,
    output_dir,
    save_filename='target_distribution_with_transforms',
    include_transformations=True
)

print("\n✅ Grafici con trasformazioni salvati")
print("\n💡 Suggerimento: Se la distribuzione log-trasformata è più normale,")
print("   considera di usare log(target) nel training del modello.")

## 6️⃣ Correlazioni con Target

In [ ]:
# Analisi correlazioni
corr_df = analyze_correlations(
    df, 
    target_col, 
    output_dir,
    threshold=0.1
)

# Top correlazioni
print("\n🔝 Top 15 correlazioni (valore assoluto):")
corr_df.head(15)

In [ ]:
# Heatmap delle top 20 correlazioni
create_correlation_heatmap(
    df,
    target_col,
    top_n=20,
    output_dir=output_dir,
    save_filename='correlation_heatmap_top20'
)

print("✅ Heatmap salvata in eda_outputs/")

## 7️⃣ Summary per Gruppi

In [ ]:
# Analisi per gruppi chiave
print("👥 ANALISI SUMMARY PER GRUPPI")
print("="*60)

group_cols = ['AI_ZonaOmi', 'AI_IdCategoriaCatastale']

for group_col in group_cols:
    if group_col not in df.columns:
        print(f"\n⚠️  Colonna {group_col} non trovata, skip")
        continue
    
    print(f"\n📊 Summary per: {group_col}")
    print("-" * 50)
    
    valid_data = df[[group_col, target_col]].dropna()
    if len(valid_data) == 0:
        print(f"❌ Nessun dato valido per {group_col}")
        continue
    
    group_stats = valid_data.groupby(group_col)[target_col].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(2)
    
    group_stats = group_stats.sort_values('count', ascending=False)
    group_stats['cv'] = (group_stats['std'] / group_stats['mean'] * 100).round(2)
    
    print(f"Gruppi trovati: {len(group_stats)}")
    print(f"\n🔝 Top 10 gruppi per dimensione:")
    display(group_stats.head(10))
    
    # Salva su CSV
    output_file = output_dir / f'group_summary_{group_col}.csv'
    group_stats.to_csv(output_file)
    print(f"💾 Salvato: {output_file}")

## 8️⃣ Check Geospaziale

In [ ]:
# Check rapido colonne geospaziali
print("🗺️  CHECK GEOSPAZIALE RAPIDO")
print("="*60)

geo_patterns = [
    'lat', 'latitude', 'lon', 'lng', 'longitude',
    'geometry', 'wkt', 'geojson', 'coord',
    'posizione', 'location'
]

geo_candidates = []
for col in df.columns:
    col_lower = col.lower()
    for pattern in geo_patterns:
        if pattern in col_lower:
            sample_vals = df[col].dropna().head(3).tolist()
            geo_candidates.append({
                'Colonna': col,
                'Tipo': str(df[col].dtype),
                'Pattern': pattern,
                'NonNull': int(df[col].count()),
                'Esempio': str(sample_vals[0])[:50] if sample_vals else 'N/A'
            })
            break

if geo_candidates:
    geo_df = pd.DataFrame(geo_candidates)
    print(f"\n🗺️  Trovate {len(geo_df)} colonne candidate geospaziali:")
    display(geo_df)
    
    # Salva
    geo_df.to_csv(output_dir / 'geospatial_columns_check.csv', index=False)
    print(f"\n💾 Salvato in {output_dir}/geospatial_columns_check.csv")
else:
    print("\n❌ Nessuna colonna geospaziale candidata trovata")

## 9️⃣ Riepilogo Finale

In [ ]:
print("\n" + "="*60)
print("🎉 ANALISI ESPLORATIVA BASIC COMPLETATA")
print("="*60)

print(f"\n📊 Dataset analizzato:")
print(f"  • Dimensioni: {df.shape[0]:,} righe × {df.shape[1]} colonne")
print(f"  • Target: {target_col}")
print(f"  • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n💾 File generati in {output_dir}/:")
output_files = list(output_dir.glob('*'))
for f in sorted(output_files):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f"  • {f.name} ({size_kb:.1f} KB)")

print(f"\n✅ Analisi completata con successo!")
print(f"\n💡 Per analisi avanzate, vedi: eda_advanced.ipynb")